# Polynomial regression

# Importing packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [2]:
frequencies = np.array([10, 12.5, 16, 20, 25, 31.5, 40, 50, 63, 80, 100,
                        125, 160, 200, 250, 315, 400, 500, 630, 800, 1000,
                        1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300,
                        8000, 10000])
windspeeds = np.array([8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13,
                       13.5, 14, 14.5, 15, 15.5, 16])
log_fr = np.log(frequencies)
log_frequencies = log_fr.reshape(-1, 1)

In [3]:
with open('turbine_1_measured.csv', 'r') as f:
    volume_measured_data = np.genfromtxt(f, delimiter=',')

In [4]:
with open('turbine_1_simulated.csv', 'r') as f:
    volume_simulated_data = np.genfromtxt(f, delimiter=',')

In [5]:
with open('turbine_2_measured.csv', 'r') as f:
    volume_measured_2_data = np.genfromtxt(f, delimiter=',')

In [6]:
with open('turbine_2_simulated.csv', 'r') as f:
    volume_simulated_2_data = np.genfromtxt(f, delimiter=',')

In [7]:
no_of_rows = volume_measured_data.shape[0]
coef_measured = np.zeros((no_of_rows,3))
coef_simulated = np.zeros((no_of_rows,3))

In [8]:
no_of_rows_2 = volume_measured_2_data.shape[0]
coef_measured_2 = np.zeros((no_of_rows_2,3))
coef_simulated_2 = np.zeros((no_of_rows_2,3))

In [9]:
degree = 2
poly = PolynomialFeatures(degree)
Xpoly = poly.fit_transform(log_frequencies)

In [10]:
for i in range(no_of_rows):
    volume_measured = volume_measured_data[i, :].reshape(-1, 1)
    volume_simulated = volume_simulated_data[i, :].reshape(-1, 1)
    
    lr1 = LinearRegression(fit_intercept=False)
    reg1 = lr1.fit(Xpoly, volume_measured)
    
    lr2 = LinearRegression(fit_intercept=False)
    reg2 = lr2.fit(Xpoly, volume_simulated)
    
    coef_measured[i,:] = lr1.coef_
    coef_simulated[i,:] = lr2.coef_

In [11]:
for i in range(no_of_rows_2):
    volume_measured_2 = volume_measured_2_data[i, :].reshape(-1, 1)
    volume_simulated_2 = volume_simulated_2_data[i, :].reshape(-1, 1)
    
    lr3 = LinearRegression(fit_intercept=False)
    reg3 = lr3.fit(Xpoly, volume_measured_2)
    
    lr4 = LinearRegression(fit_intercept=False)
    reg4 = lr4.fit(Xpoly, volume_simulated_2)
    
    coef_measured_2[i,:] = lr3.coef_
    coef_simulated_2[i,:] = lr4.coef_

In [12]:
shift_measured = np.zeros((no_of_rows,3))
shift_simulated = np.zeros((no_of_rows,3))

shift_measured[:,0] = coef_measured[:,0]
shift_simulated[:,0] = coef_simulated[:,0]

shift_measured[:,1] = -0.5 * (coef_measured[:,1] / coef_measured[:,0]) 
shift_simulated[:,1] = -0.5 * (coef_simulated[:,1] / coef_simulated[:,0])

shift_measured[:,2] = coef_measured[:,2] - (shift_measured[:,0] * shift_measured[:,1] ** 2)
shift_simulated[:,2] = coef_simulated[:,2] - (shift_simulated[:,0] * shift_simulated[:,1] ** 2)

In [13]:
shift_measured_2 = np.zeros((no_of_rows_2,3))
shift_simulated_2 = np.zeros((no_of_rows_2,3))

shift_measured_2[:,0] = coef_measured_2[:,0]
shift_simulated_2[:,0] = coef_simulated_2[:,0]

shift_measured_2[:,1] = -0.5 * (coef_measured_2[:,1] / coef_measured_2[:,0]) 
shift_simulated_2[:,1] = -0.5 * (coef_simulated_2[:,1] / coef_simulated_2[:,0])

shift_measured_2[:,2] = coef_measured_2[:,2] - (shift_measured_2[:,0] * shift_measured_2[:,1] ** 2)
shift_simulated_2[:,2] = coef_simulated_2[:,2] - (shift_simulated_2[:,0] * shift_simulated_2[:,1] ** 2)

In [14]:
weights = np.zeros((no_of_rows,3))

weights[:,0] = np.log(shift_simulated[:,0]/shift_measured[:,0])
weights[:,1] = shift_simulated[:,1] - shift_measured[:,1]
weights[:,2] = shift_simulated[:,2] - shift_measured[:,2]

In [15]:
weights_2 = np.zeros((no_of_rows_2,3))

weights_2[:,0] = np.log(shift_simulated_2[:,0]/shift_measured_2[:,0])
weights_2[:,1] = shift_simulated_2[:,1] - shift_measured_2[:,1]
weights_2[:,2] = shift_simulated_2[:,2] - shift_measured_2[:,2]

In [16]:
stack_weights = np.vstack((weights,weights_2))

In [17]:
scaler_1 = StandardScaler()
abs_weights_1 = np.absolute(weights)
abs_scaled_weights_1 = scaler_1.fit_transform(abs_weights_1)
scores_1 = np.mean(abs_scaled_weights_1, 1)

In [18]:
scaler_2 = StandardScaler()
abs_weights_2 = np.absolute(weights_2)
abs_scaled_weights_2 = scaler_2.fit_transform(abs_weights_2)
scores_2 = np.mean(abs_scaled_weights_2, 1)

In [19]:
scaler_stack = StandardScaler()
abs_weights_stack = np.absolute(stack_weights)
abs_scaled_weights_stack = scaler_stack.fit_transform(abs_weights_stack)
combined_scores_stack = np.mean(abs_scaled_weights_stack, 1)
scores_stack, scores_2_stack = np.split(combined_scores_stack, [no_of_rows])

In [20]:
scores_1.reshape(-1,1)

array([[-0.39993153],
       [ 0.53288827],
       [ 1.5774486 ],
       [ 1.37370282],
       [ 0.54748981],
       [-0.308369  ],
       [-0.75831111],
       [-1.29177236],
       [-1.28853497],
       [-1.1841821 ],
       [-0.95587346],
       [-0.40750774],
       [ 0.56541506],
       [ 2.04132411],
       [ 0.16445324],
       [-0.12759995],
       [-0.0806397 ]])

In [21]:
scores_stack.reshape(-1,1)

array([[ 0.26295637],
       [ 1.2782613 ],
       [ 2.4124367 ],
       [ 2.19141764],
       [ 1.29072053],
       [ 0.36316439],
       [-0.1168538 ],
       [-0.68371067],
       [-0.69090733],
       [-0.58158927],
       [-0.34192926],
       [ 0.24007116],
       [ 1.28412552],
       [ 2.88818024],
       [ 0.84979304],
       [ 0.53726543],
       [ 0.58597422]])

In [22]:
scores_2.reshape(-1,1)

array([[-0.58467459],
       [-1.24589674],
       [-1.61160362],
       [-1.68952129],
       [-0.6141798 ],
       [-1.01384173],
       [-0.38834794],
       [-0.846825  ],
       [-0.21078444],
       [ 0.39479745],
       [-0.10033054],
       [ 0.87545909],
       [ 1.38263431],
       [ 1.25232455],
       [ 0.84655794],
       [ 0.54599351],
       [ 1.62828836],
       [ 0.9655554 ],
       [ 0.41439507]])

In [23]:
scores_2_stack.reshape(-1,1)

array([[-0.7036477 ],
       [-0.8218475 ],
       [-0.87331166],
       [-0.9016216 ],
       [-0.7357751 ],
       [-0.80457467],
       [-0.69872911],
       [-0.76460271],
       [-0.65408465],
       [-0.55216816],
       [-0.63803845],
       [-0.47462805],
       [-0.39011049],
       [-0.40978465],
       [-0.47547108],
       [-0.5241091 ],
       [-0.34896662],
       [-0.45430348],
       [-0.54360143]])

In [24]:
minmax_1 = MinMaxScaler()
abs_weights_minmax_1 = np.absolute(weights)
abs_scaled_weights_minmax_1 = minmax_1.fit_transform(abs_weights_minmax_1)
scores_minmax_1 = np.mean(abs_scaled_weights_minmax_1, 1)

In [25]:
minmax_2 = MinMaxScaler()
abs_weights_minmax_2 = np.absolute(weights_2)
abs_scaled_weights_minmax_2 = minmax_2.fit_transform(abs_weights_minmax_2)
scores_minmax_2 = np.mean(abs_scaled_weights_minmax_2, 1)

In [26]:
minmax_stack = MinMaxScaler()
abs_weights_minmax_stack = np.absolute(stack_weights)
abs_scaled_weights_minmax_stack = minmax_stack.fit_transform(abs_weights_minmax_stack)
combined_scores_minmax_stack = np.mean(abs_scaled_weights_minmax_stack, 1)
scores_minmax_stack, scores_2_minmax_stack = np.split(combined_scores_minmax_stack, [no_of_rows])

In [27]:
scores_minmax_1.reshape(-1,1)

array([[0.2790487 ],
       [0.55381628],
       [0.86041902],
       [0.80067688],
       [0.55688439],
       [0.30620603],
       [0.177053  ],
       [0.02467393],
       [0.02156197],
       [0.05083044],
       [0.11497776],
       [0.27122149],
       [0.55216887],
       [0.98502741],
       [0.43507648],
       [0.35111275],
       [0.36405621]])

In [28]:
scores_minmax_stack.reshape(-1,1)

array([[0.30312228],
       [0.56977139],
       [0.86706774],
       [0.80915203],
       [0.5724734 ],
       [0.32951829],
       [0.20492911],
       [0.05810151],
       [0.05415825],
       [0.08223789],
       [0.14380163],
       [0.29425277],
       [0.56562109],
       [0.98519907],
       [0.45223306],
       [0.37122067],
       [0.3835813 ]])

In [29]:
scores_minmax_2.reshape(-1,1)

array([[0.34140549],
       [0.14402209],
       [0.03575192],
       [0.01151773],
       [0.33104754],
       [0.21172609],
       [0.39825499],
       [0.26235876],
       [0.45218777],
       [0.6327099 ],
       [0.4848163 ],
       [0.77568674],
       [0.92685904],
       [0.88810401],
       [0.76723895],
       [0.67773207],
       [1.        ],
       [0.80276399],
       [0.63866785]])

In [30]:
scores_2_minmax_stack.reshape(-1,1)

array([[0.05161503],
       [0.02146591],
       [0.00808254],
       [0.00113733],
       [0.04387317],
       [0.02631901],
       [0.05338182],
       [0.03629984],
       [0.06450957],
       [0.09058597],
       [0.06869472],
       [0.11051071],
       [0.13214254],
       [0.12707815],
       [0.11023888],
       [0.09776501],
       [0.1426907 ],
       [0.11563782],
       [0.09272815]])

In [31]:
np.savetxt("scores2_minmax_ns.csv", scores_minmax_2, delimiter=",")